In [3]:
#nantong's osm data for the liu big result 
import osmium as osm
import pandas as pd
import numpy as np 

class OSMHandler(osm.SimpleHandler):
    def __init__(self):
        osm.SimpleHandler.__init__(self)
        self.osm_data = []
        self.osm_node_raw = []
        self.osm_node_data  = []
        self.osm_way_data = []
        self.osm_way_raw = []
        self.osm_relation_data = []
        self.osm_relation_raw = []
        self.dict_way_id = dict()
        self.dict_node_id = dict()
        self.dict_relation_id = dict()
        

    def tag_inventory(self, elem, elem_type):
        data = dict()
        data['elem_type'] = elem_type 
        data['elem_id'] = elem.id
        data['elem_version'] = elem.version 
        data['timestemp'] = pd.Timestamp(elem.timestamp)
        data['elem_uid'] = elem.uid 
        data['elem_user'] = elem.user 
        data['elem_changeset'] = elem.changeset
        data['elem_tags'] = dict(elem.tags)
#         data['elem_location']= elem.location 
        if elem_type=='node':
            self.dict_node_id[elem.id] = len(self.osm_node_data) 
            self.osm_node_data.append(data)
#             self.osm_node_raw.append(elem)
           
        elif elem_type == 'way':
            self.dict_way_id[elem.id] = len(self.osm_way_data) 
            data['elem_nodes'] = [en.ref for en in list(elem.nodes)]
            self.osm_way_data.append(data)
#             self.osm_way_raw.append(elem)
        
        elif elem_type == 'relation':
#             print(elem)
            self.dict_relation_id[elem.id] = len(self.osm_relation_data) 
            data['elem_members'] = [ em.ref for em in  list(elem.members)]
            data['elem_members_type'] = [ em.type for em in list(elem.members)]
#             self.osm_relation_raw.append(elem)
            self.osm_relation_data.append(data) 
        
#         for tag in elem.tags:
            
#             data['elem_tag_s'] = tag
#             self.osm_data.append([elem_type, 
#                                    elem.id, 
# #                                    elem.lat,
# #                                    elem.lon,
#                                    elem.version,
#                                    pd.Timestamp(elem.timestamp),
#                                    elem.uid,
#                                    elem.user,
#                                    elem.changeset,
#                                    elem.tags,
#                                    tag.k,
#                                    tag.v])

    def node(self, n):
        self.tag_inventory(n, "node")

    def way(self, w):
        self.tag_inventory(w, "way")

    def relation(self, r):
        self.tag_inventory(r, "relation")
osmhandler = OSMHandler()
# scan the input file and fills the handler list accordingly
osmhandler.apply_file("./data/nantong.osm")
nodes = osmhandler.osm_node_data
ways = osmhandler.osm_way_data
relations = osmhandler.osm_relation_data

def get_node_index(idx):
    return osmhandler.dict_node_id[idx]
def get_way_index(idx):
    return osmhandler.dict_way_id[idx]
def get_trijectory(elem):
    ways_members = elem['elem_members']
    ways = []
    paths = []
    for wm in ways_members:
#         print(wm)
        if  wm in osmhandler.dict_way_id :
#         print(osmhandler.dict_way_id.keys())
            way = osmhandler.osm_way_data[osmhandler.dict_way_id[wm]]
#             print(way)
            path = []
            for i in way['elem_nodes']:
                if i in osmhandler.dict_node_id:
#                     node = osmhandler.osm_node_data[osmhandler.dict_node_id[i]]
                    path.append(osmhandler.dict_node_id[i])
            paths.append(path)
    return paths


# 连接矩阵用来制作拉普拉斯矩阵 为GCN做准备 但是这个里面太大了 我的电脑跑不起来 有几百万参数 所以我不使用这个 由于我的过程比较少，所以我选择过程作为矩阵也就是下面那个函数  
def get_ways_other(elem):
    nodes =  elem[elem_nodes]
    path = []
    for i in nodes:
        if i in osmhandler.dict_node_id:
            path.append(osmhandler.dict_node_id[i])
    return path 
        
def get_ways(elem):
    ways_members = elem['elem_members']
    ways = []
    paths = []
    for wm in ways_members:
#         print(wm)
        if  wm in osmhandler.dict_way_id :
#         print(osmhandler.dict_way_id.keys())
            way = osmhandler.osm_way_data[osmhandler.dict_way_id[wm]]
#             print(way)
            path = []
            for i in way['elem_nodes']:
                if i in osmhandler.dict_node_id:
#                     node = osmhandler.osm_node_data[osmhandler.dict_node_id[i]]
                    path.append(osmhandler.dict_node_id[i])
            paths.append(path)

In [4]:
node_size = len(nodes)
way_size = len(ways)
relation_size  = len(relations)
#这里是小样本的轨迹数据
print(node_size)
print(way_size)
print(relation_size)

324020
53002
964


In [5]:
allpath = []
for i in range(len(relations)):
    allpath += get_trijectory(relations[i])

In [6]:

import scipy.sparse as sp
S = sp.dok_matrix((node_size, node_size), dtype=np.float32)

linjie = sp.dok_matrix((node_size, node_size), dtype=np.float32)

for path in allpath:
    for i in range(1,len(path)):
        S[path[i-1],path[i]] += 1 
        S[path[i],path[i-1]] += 1 
        if linjie.get((path[i-1],path[i]))==0:
            linjie[path[i-1],path[i]] = 1
            linjie[path[i],path[i-1]] = 1 
       
    
from sklearn.decomposition import TruncatedSVD
clf = TruncatedSVD(128)
S_pca = clf.fit_transform(S)

# 所以在这里 其中linjie不可避免的成为了对应的连通性矩阵
# 同时S_pca 成了节点轨迹矩阵的 也就作为了GCN的属性矩阵


In [8]:
from scipy import io
# sp.save_npz("node_property.npz",S)
# sp.save_npz("linjie.npz",linjie)
io.mmwrite("node_property.mtx",S_pca)
io.mmwrite("linjie.mtx",linjie)
io.mmwrite('source_link.mtx',S)

In [32]:
io.mmwrite("linjie.mtx",S_pca)

In [10]:
io.mmwrite('source_link.mtx',S)

array([-2.1537559e-07, -9.7161808e-07, -5.2053650e-07, -4.0266077e-06,
       -1.2470543e-06, -1.1152174e-06,  4.2225397e-06, -1.5435741e-06,
       -1.6860404e-06,  3.7684240e-06, -3.2563448e-06,  6.1976829e-07,
        2.9879641e-06, -4.2938386e-06, -2.4882888e-07,  5.9274058e-08,
        1.7290970e-06, -3.6829351e-06, -4.4181284e-06, -3.3271908e-07,
       -1.3050508e-06,  2.3761324e-06, -6.1176597e-06,  4.7389158e-06,
        3.7659549e-06, -1.8235064e-06,  3.3262987e-07, -2.9220746e-06,
       -3.5361290e-06,  3.4514130e-06, -4.1965700e-06,  3.6689525e-07,
       -1.3131113e-06, -1.3356573e-06, -1.7811311e-07, -2.3679296e-07,
        1.0450824e-06, -1.9495085e-06,  2.0360549e-06, -1.0097391e-06,
        2.0824989e-06, -2.1026950e-07,  1.8215444e-06, -2.1705655e-06,
       -1.1627476e-06,  1.4232653e-06,  2.8135688e-07, -9.7675832e-07,
       -6.9369935e-07, -1.1636102e-07,  1.3760906e-06,  1.9925674e-06,
       -6.2540110e-08, -1.0520297e-06, -2.3678908e-06,  1.0606960e-06,
      

In [5]:
print(nodes[0])
print(ways[0])


{'elem_type': 'node', 'elem_id': 28398853, 'elem_version': 1, 'timestemp': Timestamp('2007-05-01 21:51:49+0000', tz='UTC'), 'elem_uid': 5553, 'elem_user': 'dkt', 'elem_changeset': 36445, 'elem_tags': {'created_by': 'almien_coastlines', 'source': 'PGS'}}
{'elem_type': 'way', 'elem_id': 4550743, 'elem_version': 6, 'timestemp': Timestamp('2014-10-22 20:49:28+0000', tz='UTC'), 'elem_uid': 2226712, 'elem_user': 'dannykath', 'elem_changeset': 26265507, 'elem_tags': {'source': 'PGS & Bing', 'waterway': 'riverbank'}}


In [35]:
print(relations[0])

{'elem_type': 'relation', 'elem_id': 53644, 'elem_version': 6, 'timestemp': Timestamp('2018-04-14 13:51:08+0000', tz='UTC'), 'elem_uid': 110639, 'elem_user': 'aighes', 'elem_changeset': 58089649, 'elem_tags': {'type': 'multipolygon', 'waterway': 'riverbank'}, 'elem_members': [osmium.osm.RelationMember(ref=579858789, type='w', role='outer'), osmium.osm.RelationMember(ref=579858797, type='w', role='outer'), osmium.osm.RelationMember(ref=579858796, type='w', role='outer'), osmium.osm.RelationMember(ref=579858794, type='w', role='outer'), osmium.osm.RelationMember(ref=579858793, type='w', role='outer'), osmium.osm.RelationMember(ref=579858791, type='w', role='outer'), osmium.osm.RelationMember(ref=579858790, type='w', role='outer'), osmium.osm.RelationMember(ref=579858788, type='w', role='outer'), osmium.osm.RelationMember(ref=579858787, type='w', role='outer'), osmium.osm.RelationMember(ref=579858799, type='w', role='outer'), osmium.osm.RelationMember(ref=326666038, type='w', role='inner'

In [7]:
ways[0]

{'elem_type': 'way',
 'elem_id': 4550743,
 'elem_version': 6,
 'timestemp': Timestamp('2014-10-22 20:49:28+0000', tz='UTC'),
 'elem_uid': 2226712,
 'elem_user': 'dannykath',
 'elem_changeset': 26265507,
 'elem_tags': {'source': 'PGS & Bing', 'waterway': 'riverbank'},
 'elem_nodes': [osmium.osm.NodeRef(ref=3, location=osmium.osm.Location(x=-1, y=-1)),
  osmium.osm.NodeRef(ref=164, location=osmium.osm.Location(x=0, y=0)),
  osmium.osm.NodeRef(ref=0, location=osmium.osm.Location(x=0, y=0)),
  osmium.osm.NodeRef(ref=0, location=osmium.osm.Location(x=3285157, y=0)),
  osmium.osm.NodeRef(ref=1, location=osmium.osm.Location(x=27996256, y=1)),
  osmium.osm.NodeRef(ref=24, location=osmium.osm.Location(x=53225474, y=-740114989)),
  osmium.osm.NodeRef(ref=228, location=osmium.osm.Location(x=0, y=0)),
  osmium.osm.NodeRef(ref=8385530335628260720, location=osmium.osm.Location()),
  osmium.osm.NodeRef(ref=7957695011148363322, location=osmium.osm.Location(x=0, y=0)),
  osmium.osm.NodeRef(ref=1, locat

In [ ]:
get_trijectory(relations[0])

In [29]:
osmhandler.dict_way_id.keys()

dict_keys([2226712, 42537, 1804331, 2286161, 17497, 534149, 257555, 1832883, 5553, 6591408, 2215592, 23785, 2500, 2748195, 4816761, 2219338, 304705, 9176, 87483, 4220901, 413914, 476483, 3182193, 386131, 1185631, 2512300, 5317567, 42429, 953424, 83557, 388809, 3189456, 1380023, 66811, 253683, 421504, 14002, 1829683, 616774, 499500, 3731755, 720609, 2511706, 510836, 3164012, 1240849, 110639, 2377377, 1330847, 1263561, 4803527, 4803512, 4803528, 8350466, 5888135, 6052789, 1236135, 2507347, 384084, 2891170, 2508151, 1937363, 5891, 1708958, 3091009, 2905914, 5509234, 26299, 2554698, 396203, 437598, 3057995, 1911765, 1051550, 5996162, 597625, 5147743, 6714381, 4803525, 2665933, 6127520, 224440, 1901087, 2728141, 2513563, 4497723, 1803854, 3619022, 3619060, 7289974, 7755848, 7595195, 2948558, 1795336, 4210803, 2823295, 5400428, 5400480, 2728060, 3642735, 665748, 3775202, 3424135, 527721, 161619, 1293194, 172147, 967832, 2046201, 632124, 497849, 29084, 5460325, 454589, 2489224, 91490, 8570285

In [28]:
get_trijectory(relations[1])

483517995


KeyError: 483517995

In [17]:
list(relations_raw[1].members)[2]

osmium.osm.RelationMember(ref=483518005, type='w', role='side_stream')

{'elem_type': 'way',
 'elem_id': 4550743,
 'elem_version': 6,
 'timestemp': Timestamp('2014-10-22 20:49:28+0000', tz='UTC'),
 'elem_uid': 2226712,
 'elem_user': 'dannykath',
 'elem_changeset': 26265507,
 'elem_tags': {'source': 'PGS & Bing', 'waterway': 'riverbank'}}

579858789

In [42]:
#这里是小样本的轨迹数据
print(len(nodes))
print(len(ways))
print(len(relation))

324020
53002
964


In [20]:
class trajectory2graph:
    def __init__(self,node_data,ways,trajectory):
        

In [30]:
temp[30000]

['way',
 189083399,
 6,
 True,
 Timestamp('2015-11-21 04:19:03+0000', tz='UTC'),
 2215592,
 'xiaotu',
 35479866,
 3,
 'highway',
 'tertiary']